### `done`
given a topic
- can generate script
- can generate images for script

### `pending`
Having script images
- should generate speech
- should generate music
- combine speech and images into video
- add music


In [4]:
from langchain.chains import LLMChain, SequentialChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate

In [5]:
llm = ChatGoogleGenerativeAI(model='gemini-pro')

In [6]:

template1 = """
You are a content writer for a prominent YouTuber. Given a specific topic, you will generate both text for image creation and text for voice-over narration.

**Instructions:**

1. **Generate Text for Image Creation:** 
   - Generate sentences from the provided topic that are visually descriptive and capture the main ideas or scenes. These sentences will guide the creation of compelling images.
   - Ensure each sentence is concise, vivid, and clearly depicts a scene or idea that can be visualized as an image. If characters are involved, describe them consistently in terms of age, gender, etc.

2. **Generate Text for Voice-Over:** 
   - Generate sentences from the provided topic and the image text that can be used for voice-over narration. These sentences should be engaging, suitable for narration.
   - Ensure each sentence is concise, clear, and crafted to capture the viewer's interest, effectively conveying the main points or scenes from the topic.
   - Make sure that each sentence is long enough to narrate for 10 seconds
   
3. **Ensure Synergy between Image and Voice-Over:**
   - Ensure that each point in the image creation text corresponds logically and thematically with the corresponding point in the voice-over text, maintaining coherence throughout the video content.

**Topic:**  
{topic}

Output format:

**Title:** title  
**Description:** description  
**Tags:** tag1, tag2, tag3

**Sentences for Image Generation:**
1. sentence-1
2. sentence-2
3. sentence-3
and so on

**Sentences for Voice-Over:**
1. sentence-1
2. sentence-2
3. sentence-3
and so on"""

template2 = """You are an content writer working for a Youtuber. Given a script, you will modify the script according to the instructions given below.

**Instructions:**
The script contains Title, Description, Tags, Sentences for Image Generation, Sentences for Voice-Over.
- Ensure that each point in the 'Sentences for Image Generation' corresponds logically and thematically with the corresponding point in the 'Sentences for Voice-Over', maintaining coherence throughout the video content.
- For 'Sentences for Image Generation' add a point in the beginning, that has Visual description related to the hook.
- For 'Sentences for Voice-Over' add a line related to the hook at the beginning that attracts audience attention and a line related to the Call-to-action - prompting viewers to like, comment, and subscribe at the end.
- These sentences should be engaging, suitable for narration.

**Script:**  
{script}

Make sure that number of sentences in 'Sentences for Voice-Over' are one more than number of sentences in 'Sentences for Image Generation'. if not modify script."""

prompt1 = PromptTemplate.from_template(template1)
prompt2 = PromptTemplate.from_template(template2)

In [7]:
topic = '5 Greatest kings of all time'

In [8]:
chain1 = LLMChain(llm=llm, prompt=prompt1, output_key='script')
chain2 = LLMChain(llm=llm, prompt=prompt2, output_key='output')
# chain3 = LLMChain(llm=self.llm, prompt=text_speech_prompt, output_key='text_speech')
overall_chain = SequentialChain(
chains=[chain1, chain2],#, chain3],
input_variables=["topic"],
output_variables=["script", "output"])#, "text_img", "text_speech"])
import langchain
langchain.debug = True
output = overall_chain.invoke(topic)
langchain.debug = False

c:\Users\prudh\anaconda3\envs\gen_ai1\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


[chain/start] [chain:SequentialChain] Entering Chain run with input:
{
  "topic": "5 Greatest kings of all time"
}
[chain/start] [chain:SequentialChain > chain:LLMChain] Entering Chain run with input:
{
  "topic": "5 Greatest kings of all time"
}
[llm/start] [chain:SequentialChain > chain:LLMChain > llm:ChatGoogleGenerativeAI] Entering LLM run with input:
{
  "prompts": [
    "Human: \nYou are a content writer for a prominent YouTuber. Given a specific topic, you will generate both text for image creation and text for voice-over narration.\n\n**Instructions:**\n\n1. **Generate Text for Image Creation:** \n   - Generate sentences from the provided topic that are visually descriptive and capture the main ideas or scenes. These sentences will guide the creation of compelling images.\n   - Ensure each sentence is concise, vivid, and clearly depicts a scene or idea that can be visualized as an image. If characters are involved, describe them consistently in terms of age, gender, etc.\n\n2. 

In [9]:
print(output['script'])

**Title:** The Five Kings Who Ruled the World

**Description:** Join us as we explore the lives and legacies of the five greatest kings of all time. From ancient Egypt to the British Empire, these monarchs shaped the course of history and left an unforgettable mark on the world.

**Tags:** history, kings, leadership, power, legacy

**Sentences for Image Creation:**

1. A majestic pharaoh sits on his golden throne, adorned with intricate headdresses and a regal scepter.
2. A young Alexander the Great leads his mighty army into battle, his eyes gleaming with determination.
3. Julius Caesar stands on the steps of the Roman Senate, his eloquent speech captivating the crowd.
4. Charlemagne, crowned as Holy Roman Emperor, presides over a grand feast in his magnificent palace.
5. Queen Victoria, a beloved monarch, rules over a vast British Empire, her reign marked by progress and prosperity.

**Sentences for Voice-Over:**

1. Throughout history, certain kings have risen above the rest, their 

In [10]:
print(output['output'])

**Title:** The Five Kings Who Ruled the World

**Description:** Join us as we explore the lives and legacies of the five greatest kings of all time. From ancient Egypt to the British Empire, these monarchs shaped the course of history and left an unforgettable mark on the world.

**Tags:** history, kings, leadership, power, legacy

**Sentences for Image Creation:**

1. **Visual description related to the hook:** A majestic pharaoh sits on his golden throne, adorned with intricate headdresses and a regal scepter.
2. A young Alexander the Great leads his mighty army into battle, his eyes gleaming with determination.
3. Julius Caesar stands on the steps of the Roman Senate, his eloquent speech captivating the crowd.
4. Charlemagne, crowned as Holy Roman Emperor, presides over a grand feast in his magnificent palace.
5. Queen Victoria, a beloved monarch, rules over a vast British Empire, her reign marked by progress and prosperity.

**Sentences for Voice-Over:**

1. **Line related to the h

In [74]:
type(output['output'])

str

## Trying new models

In [26]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
import torch

In [32]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype='float16',
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    "NousResearch/Hermes-2-Pro-Llama-3-8B",
    quantization_config=bnb_config
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model-00004-of-00004.safetensors:  28%|##7       | 325M/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

In [35]:
model.save_pretrained('./llama')

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Hermes-2-Pro-Llama-3-8B")
tokenizer.save_pretrained("./llama")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


('./llama\\tokenizer_config.json',
 './llama\\special_tokens_map.json',
 './llama\\tokenizer.json')

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the path to the local directory where the model is saved
local_model_path = "./llama"

# Load the model from the local directory
model = AutoModelForCausalLM.from_pretrained(local_model_path)

# (Optional) Load the tokenizer from the local directory
tokenizer = AutoTokenizer.from_pretrained(local_model_path)

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
input_text = """"Given the following YouTube Shorts script and topic, extract key sentences that can be used to generate images. The sentences should be visually descriptive and capture the main ideas or scenes from the script.

# **Script:**
# {script}

**Topic:**
{Indian Gods}

Note: count of sentences for output depends on topic
**Sentences for Image Generation:**
1. [Visual description related to the hook]
2. [Visual description related to Main Content Part 1]
3. [Visual description related to Main Content Part 2]
.
.
n. [Visual description related to the call-to-action]

Note: The output should contain only the points 
Make sure each sentence is concise and vivid, clearly depicting a scene or idea that can be visualized as an image. If any characters are preset in the text, ensure all points describe characters with consistent characteristics such as age, gender, etc."""

# Encode the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Generate output
outputs = model.generate(inputs["input_ids"])

# Decode the output text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True, max_new_length=200)

# Print the generated response
print(generated_text)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Define the path to the local directory where the model is saved
# local_model_path = "./llama"

# # Load the model from the local directory
# model = AutoModelForCausalLM.from_pretrained(local_model_path)

# # Load the tokenizer from the local directory
# tokenizer = AutoTokenizer.from_pretrained(local_model_path)

# Prepare some input text
input_text = "Hello, how are you?"

# Encode the input text
inputs = tokenizer(input_text, return_tensors="pt")

# Generate output with specific parameters
outputs = model.generate(
    inputs["input_ids"],
    max_new_tokens=200,  # Control the maximum number of new tokens generated
    num_return_sequences=1,  # Number of sequences to return
    no_repeat_ngram_size=2,  # No repetition of n-grams
    early_stopping=True  # Stop early if the end-of-sequence token is produced
)

# Decode the output text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated response
print(generated_text)


In [4]:
import subprocess

def video_create(image, audio, num):
    try:
        ffmpeg_command = [
            'ffmpeg',
            '-loop', '1',
            '-i', image,
            '-i', audio,
            '-filter_complex', '[0:v]fade=t=in:st=0:d=2[v0];[v0][1:a]concat=n=2:v=1:a=1[v][a]',
            '-map', '[v]',
            '-map', '[a]',
            '-c:v', 'libx264',
            '-c:a', 'aac',
            '-b:a', '192k',
            '-shortest',
            f'outputs/temp_videos/video_{num}.mp4'
        ]
        
        result = subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
        print(result.stdout)
        
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg command failed with error code {e.returncode}:")
        print(e.stderr)

# Example usage
video_create('outputs/images/image_0.jpg', 'outputs/speech/speech_0.mp3', 111)


FFmpeg command failed with error code 4294967274:
ffmpeg version 2024-05-20-git-127ded5078-full_build-www.gyan.dev Copyright (c) 2000-2024 the FFmpeg developers
  built with gcc 13.2.0 (Rev5, Built by MSYS2 project)
  configuration: --enable-gpl --enable-version3 --enable-static --disable-w32threads --disable-autodetect --enable-fontconfig --enable-iconv --enable-gnutls --enable-libxml2 --enable-gmp --enable-bzlib --enable-lzma --enable-libsnappy --enable-zlib --enable-librist --enable-libsrt --enable-libssh --enable-libzmq --enable-avisynth --enable-libbluray --enable-libcaca --enable-sdl2 --enable-libaribb24 --enable-libaribcaption --enable-libdav1d --enable-libdavs2 --enable-libuavs3d --enable-libxevd --enable-libzvbi --enable-librav1e --enable-libsvtav1 --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxavs2 --enable-libxeve --enable-libxvid --enable-libaom --enable-libjxl --enable-libopenjpeg --enable-libvpx --enable-mediafoundation --enable-libass --enable-frei0r --e

In [6]:
import subprocess

def video_add_effects(input_video, output_video):
    try:
        ffmpeg_command = [
            'ffmpeg',
            '-i', input_video,
            '-vf', 'fade=in:st=0:d=2',
            '-c:v', 'libx264',
            '-c:a', 'aac',
            '-b:a', '192k',
            output_video
        ]
        
        result = subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
        print(result.stdout)
        
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg command failed with error code {e.returncode}:")
        print(e.stderr)

# Example usage
video_add_effects('outputs/temp_videos/video_1.mp4', 'output_video.mp4')

In [9]:
import subprocess

def add_watermark(input_video, watermark_image, output_video):
    try:
        ffmpeg_command = [
            'ffmpeg',
            '-i', input_video,
            '-i', watermark_image,
            '-filter_complex', 'overlay=W-w-1:H-h-1',  # Adjust position as needed
            '-c:v', 'libx264',
            '-c:a', 'aac',
            '-b:a', '192k',
            output_video
        ]
        
        result = subprocess.run(ffmpeg_command, check=True, capture_output=True, text=True)
        print(result.stdout)
        
    except subprocess.CalledProcessError as e:
        print(f"FFmpeg command failed with error code {e.returncode}:")
        print(e.stderr)

# Example usage:
add_watermark('outputs/final_video/video1.mp4', 'outputs/images/image_1.jpg', 'output_video.mp4')